In [ ]:
import random
import math
import pandas as pd
import numpy as np
from pandas import StringDtype
import bmll2 as b2

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import LogFormatterSciNotation
from scipy.stats import linregress
from scipy.optimize import curve_fit
from scipy.stats import t

## GRT

The code that produce the plots for the concave execution profile of metaorder impact is given below along with the code for the fitted curves. The point estimates of the variables for the fitted curve are tabulated

In [ ]:
ticker              = 'GRT'
N                   = 20
trader_distribution = 'power'
alpha               = 2
identifier          = f'{trader_distribution}_{N}'

In [ ]:
b2.get_file(f'test_data/GRT_impact_profile_{identifier}.csv')
impact_profile_GRT = pd.read_csv(f'{ticker}_impact_profile_{identifier}.csv')

In [ ]:
bins = np.linspace(0, 1, 51)
phi_bins_GRT = pd.cut(impact_profile_GRT['phi'], bins = bins)

dynamic_impact_bins_vals_GRT = (impact_profile_GRT['scaled impact']).groupby(phi_bins_GRT, observed = False).mean()
phi_bins_centers_GRT         = [interval.mid for interval in dynamic_impact_bins_vals_GRT.index.categories]

prepend = pd.Series([0], index = ['0'])
dynamic_impact_bins_vals_GRT = pd.concat([prepend, dynamic_impact_bins_vals_GRT])
phi_bins_centers_GRT.insert(0, 0)

In [ ]:
def execution_profile_model(phi, exponent, I):
    return  I * (phi ** exponent)

phi    = np.array(phi_bins_centers_GRT)
impact = np.array(dynamic_impact_bins_vals_GRT)

mask       = phi >= 0 
phi_fit    = phi[mask]
impact_fit = impact[mask]

params, covariance = curve_fit(execution_profile_model, phi_fit, impact_fit, p0 = [0.5, 0.0002])

exponent_est_GRT = params[0]
I_est_GRT        = params[1]
print('Estimated exponent:', exponent_est_GRT)
print('Estimated peak impact', I_est_GRT)

fitted_curve_GRT = execution_profile_model(phi_bins_centers_GRT, exponent_est_GRT, I_est_GRT)

n   = len(phi_fit)
p   = len(params)           # number of parameters (2 here)
dof = n - p                # degrees of freedom

alpha = 0.05              # 95% CI
tval  = t.ppf(1 - alpha / 2, dof)
se    = np.sqrt(np.diag(covariance))
interval_half_widths = tval * se[0], tval * se[1]
print('Interval half widths:', interval_half_widths)

I_ci        = I_est_GRT - tval * se[1], I_est_GRT + tval * se[1]
exponent_ci = exponent_est_GRT - tval * se[0], exponent_est_GRT + tval * se[0]
print('Scaling I 95% CI:', I_ci)
print('Exponent 95% CI:', exponent_ci)


In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))

ax.scatter(phi_bins_centers_GRT, dynamic_impact_bins_vals_GRT, color = 'olive', marker = 'o', label = 'GRT dynamic impact')
ax.set_xlabel(r'Rescaled volume time $\phi = \sum q_i/Q$', fontsize = 14)
ax.set_ylabel(r'Dynamic impact $\frac{I(\phi Q)}{\sigma_{D} \sqrt{Q}}$', fontsize = 14)
ax.plot(phi_bins_centers_GRT, fitted_curve_GRT, color = 'red', linestyle = '--', label = rf'Fitted curve: $Y \times\phi^{{{exponent_est_GRT:.2f}}}$')
ax.legend()

ax.set_box_aspect(1)
plt.tight_layout()

plt.savefig(f'{ticker}_concave_profile_{identifier}.pdf')
b2.put_file(f'{ticker}_concave_profile_{identifier}.pdf', 'figures')
b2.get_file(f'figures/{ticker}_concave_profile_{identifier}.pdf')

plt.show()

## GFI

The code that produce the plots for the concave execution profile of metaorder impact is given below along with the code for the fitted curves. The point estimates of the variables for the fitted curve are tabulated

In [ ]:
ticker              = 'GFI'
N                   = 20
trader_distribution = 'power'
alpha               = 2
identifier          = f'{trader_distribution}_{N}'

In [ ]:
b2.get_file(f'test_data/GFI_impact_profile_{identifier}.csv')
impact_profile_GFI = pd.read_csv(f'{ticker}_impact_profile_{identifier}.csv')

In [ ]:
bins = np.linspace(0, 1, 51)
phi_bins_GFI = pd.cut(impact_profile_GFI['phi'], bins = bins)

dynamic_impact_bins_vals_GFI = (impact_profile_GFI['scaled impact']).groupby(phi_bins_GFI, observed = False).mean()
phi_bins_centers_GFI         = [interval.mid for interval in dynamic_impact_bins_vals_GFI.index.categories]

prepend = pd.Series([0], index = ['0'])
dynamic_impact_bins_vals_GFI = pd.concat([prepend, dynamic_impact_bins_vals_GFI])
phi_bins_centers_GFI.insert(0, 0)

In [ ]:
def execution_profile_model(phi, exponent, I):
    return  I * (phi ** exponent)

phi    = np.array(phi_bins_centers_GFI)
impact = np.array(dynamic_impact_bins_vals_GFI)

mask       = phi >= 0 
phi_fit    = phi[mask]
impact_fit = impact[mask]

params, covariance = curve_fit(execution_profile_model, phi_fit, impact_fit, p0 = [0.5, 0.0002])

exponent_est_GFI = params[0]
I_est_GFI        = params[1]
print('Estimated exponent:', exponent_est_GFI)
print('Estimated peak impact', I_est_GFI)

fitted_curve_GFI = execution_profile_model(phi_bins_centers_GFI, exponent_est_GFI, I_est_GFI)

n   = len(phi_fit)
p   = len(params)           # number of parameters (2 here)
dof = n - p                 # degrees of freedom

alpha = 0.05                # 95% CI
tval  = t.ppf(1 - alpha / 2, dof)
se    = np.sqrt(np.diag(covariance))
interval_half_widths = tval * se[0], tval * se[1]
print('Interval half widths:', interval_half_widths)

I_ci        = I_est_GFI - tval * se[1], I_est_GFI + tval * se[1]
exponent_ci = exponent_est_GFI - tval * se[0], exponent_est_GFI + tval * se[0]
print('Scaling I 95% CI:', I_ci)
print('Exponent 95% CI:', exponent_ci)


In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))

ax.scatter(phi_bins_centers_GFI, dynamic_impact_bins_vals_GFI, color = 'olive', marker = 'o', label = 'GFI dynamic impact')
ax.set_xlabel(r'Rescaled volume time $\phi = \sum q_i/Q$', fontsize = 14)
ax.set_ylabel(r'Dynamic impact $\frac{I(\phi Q)}{\sigma_{D} \sqrt{Q}}$', fontsize = 14)
ax.plot(phi_bins_centers_GFI, fitted_curve_GFI, color = 'red', linestyle = '--', label = rf'Fitted curve: $Y \times\phi^{{{exponent_est_GFI:.2f}}}$')
ax.legend()

ax.set_box_aspect(1)
plt.tight_layout()

plt.savefig(f'{ticker}_concave_profile_{identifier}.pdf')
b2.put_file(f'{ticker}_concave_profile_{identifier}.pdf', 'figures')
b2.get_file(f'figures/{ticker}_concave_profile_{identifier}.pdf')

plt.show()


## Table of fitted values

In [ ]:
execution_concavity_table = pd.DataFrame({'': ['GFI', 'GRT'], 'scaling factor': [I_est_GFI, I_est_GRT],
                                          'exponent': [exponent_est_GFI, exponent_est_GRT]})
print(execution_concavity_table.to_latex(index = False, caption = 'Fitted values for metaorder execution profiles of GFI and GRT',
                                         label = 'tab:execution profile values'))